In [33]:
import mysql.connector
from mysql.connector import errorcode

class MySQLConnection:
    def __init__(self, host: str, user: str, password: str, database: str = None, port: int = 3306):
        self.__host = host
        self.__user = user
        self.__password = password
        self.__database = database
        self.__port = port
        self.__connection = None
        self.__cursor = None

    def connect(self):
        try:
            self.__connection = mysql.connector.connect(
                host=self.__host,
                user=self.__user,
                password=self.__password,
                database=self.__database if self.__database else None,
                port=self.__port
            )
            self.__cursor = self.__connection.cursor()
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_BAD_DB_ERROR and self.__database:
                self.create_database()
                self.__connection = mysql.connector.connect(
                    host=self.__host,
                    user=self.__user,
                    password=self.__password,
                    database=self.__database,
                    port=self.__port
                )
                self.__cursor = self.__connection.cursor()
            else:
                raise Exception(f"Error connecting to the database: {err}")

    def disconnect(self):
        try:
            if self.__connection.is_connected():
                self.__cursor.close()
                self.__connection.close()
        except mysql.connector.Error as err:
            raise Exception(f"Error disconnecting from the database: {err}")

    def create_database(self, database: str = None):
        try:
            if database: self.__database = database
            temp_connection = mysql.connector.connect(
                host=self.__host,
                user=self.__user,
                password=self.__password,
                port=self.__port
            )
            temp_cursor = temp_connection.cursor()
            temp_cursor.execute(f"CREATE DATABASE {self.__database}")
            temp_cursor.close()
            temp_connection.close()
        except mysql.connector.Error as err:
            raise Exception(f"Failed to create database: {err}")

    def create_table(self, table_name: str, columns: dict):
        try:
            self.connect()
            columns_str = ', '.join([f'{col} {data_type}' for col, data_type in columns.items()])
            create_table_query = f'CREATE TABLE {table_name} ({columns_str})'
            self.__cursor.execute(create_table_query)
            self.__connection.commit()
        except mysql.connector.Error as err:
            raise Exception(f"Failed to create table: {err}")
        finally:
            self.disconnect()

    def insert_record(self, table_name: str, record: dict):
        try:
            self.connect()
            columns = ', '.join(record.keys())
            values = tuple(record.values())
            placeholders = ', '.join(['%s'] * len(record))
            insert_query = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders})'
            self.__cursor.execute(insert_query, values)
            self.__connection.commit()
        except mysql.connector.Error as err:
            raise Exception(f"Failed to insert record: {err}")
        finally:
            self.disconnect()

    def select_record(self, table_name: str, conditions: str = None):
        try:
            self.connect()
            select_query = f'SELECT * FROM {table_name}'
            if conditions:
                select_query += f' WHERE {conditions}'
            self.__cursor.execute(select_query)
            records = self.__cursor.fetchall()
            return records
        except mysql.connector.Error as err:
            raise Exception(f"Failed to select record: {err}")
        finally:
            self.disconnect()

    def update_record(self, table_name: str, record: dict, conditions: str):
        try:
            self.connect()
            set_clause = ', '.join([f'{key}=%s' for key in record.keys()])
            values = tuple(record.values())
            update_query = f'UPDATE {table_name} SET {set_clause} WHERE {conditions}'
            self.__cursor.execute(update_query, values)
            self.__connection.commit()
        except mysql.connector.Error as err:
            raise Exception(f"Failed to update record: {err}")
        finally:
            self.disconnect()

    def delete_record(self, table_name: str, conditions: str):
        try:
            self.connect()
            delete_query = f'DELETE FROM {table_name} WHERE {conditions}'
            self.__cursor.execute(delete_query)
            self.__connection.commit()
        except mysql.connector.Error as err:
            raise Exception(f"Failed to delete record: {err}")
        finally:
            self.disconnect()


In [34]:
passwd = 'pearlbharti'
mysql_conn = MySQLConnection(host='localhost', user='root', password=passwd, database=None)
mysql_conn.connect()

In [35]:
mysql_conn.create_database("test_database")

In [36]:
columns = {
            'id': 'INT AUTO_INCREMENT PRIMARY KEY',
            'name': 'VARCHAR(255)',
            'age': 'INT'
        }
mysql_conn.create_table('users', columns)


In [37]:
record1 = {'name': 'Alice', 'age': 25}
mysql_conn.insert_record('users', record1)

In [38]:
record2 = {'name': 'Bob', 'age': 30}
mysql_conn.insert_record('users', record2)

In [39]:
records = mysql_conn.select_record('users')
print("Selected records:", records)

Selected records: [(1, 'Alice', 25), (2, 'Bob', 30)]


In [40]:
update_record = {'age': 26}
mysql_conn.update_record('users', update_record, 'name="Alice"')

In [41]:
updated_records = mysql_conn.select_record('users')
print("Updated records:", updated_records)

Updated records: [(1, 'Alice', 26), (2, 'Bob', 30)]


In [42]:
mysql_conn.delete_record('users', 'name="Bob"')

        # Select remaining records
remaining_records = mysql_conn.select_record('users')
print("Remaining records:", remaining_records)

Remaining records: [(1, 'Alice', 26)]
